```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
import matplotlib.pyplot as plt

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/signal_extraction.ipynb>`, or a {download}`python script <converted/signal_extraction.py>` with code cells.
```

# Tutorial 2: Measurement of Fluorescent Signals and Waves

This tutorial will cover the measurement of fluorescent signals and wave dynamics from cardiac optical mapping data more generally. In [Tutorial 1](basics.ipynb) we already loaded and displayed video data, performed numerical motion tracking and motion-stabilization, performed some post-processing, and displayed action potential waves. In this tutorial, we will cover the post-processing routines used to extract and display the optical signals in more detail. Note that this tutorial covers post-processing routines that can only be applied to motion-inhibited tissues. The motion-inhibition can result from either pharmacological excitation-contraction uncoupling with agents such as Blebbistatin or from numerical motion correction. Therefore, you could analyze your own video data, even if it was acquired with Blebbistatin, following the steps outlined in this tutorial. We first repeat the steps from the previous tutorial by loading the experimental data and performing motion correction. Accordingly, if you analyze videos that were acquired with Blebbistatin, you can simply skip the motion correction steps.

In [ ]:
import optimap as om
import numpy as np

filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um.npy')
# alterantively set filepath = '/folder_on_your_computer/Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um.npy'
video = om.load_video(filepath)
warped = om.motion_compensate(video,
                              contrast_kernel=5,
                              presmooth_spatial=1,
                              presmooth_temporal=1)
om.video.play(warped, title="recording without motion", skip_frame=3);

In [ ]:
# Hidden, same as above but shorten the video slightly

import optimap as om
import numpy as np

filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um.npy')
video = om.load_video(filepath, use_mmap=True)[:500]
filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um_warped.npy', silent=True)
warped = om.load_video(filepath, use_mmap=True)[:500]
render_ani_func(lambda: om.video.play(warped, title="recording without motion", skip_frame=1, interval=20))

For better visualization in the next steps we'll need a mask of the background. Here we use {func}`background_mask` to create a mask of the background from the first frame of the data using an automatic threshold.

Pixels in red are considered background (value of `True`), and pixels in blue are considered foreground (value of `False`).

In [ ]:
background_mask = om.background_mask(warped[0])

## Fluorescence wave isolation

```{warning}
This tutorial is currently work in progress. We will add more information soon.
```

We will cover two possible ways to extract the fluorescence wave dynamics:

1. Using sliding window normalization. This method conserves the shape of the action potential and normalizes the signal to [0, 1] using a sliding window. However, it can be sensitive to noise and changes of the base fluorescence due to inhomogeneous illumination when using motion correction.
2. Using a frame difference approach. This method is more robust, however it does not conserve the shape of the action potential. It is useful to visualize the propagation the front of the action potential waves.


### Sliding window normalization

The sliding/rolling window normalization performs a pixel-based normalization of the signal to [0, 1] within a short temporal window for each time point. The function {func}`video.normalize_pixelwise_slidingwindow` performs this operation, by computing the following equation for each pixel $(x, y)$ and frame $t$:

$$
\begin{align}
    \text{signal}_{\text{norm}}(t, x, y) = \frac{\text{signal}(t, x, y) - \text{min}_{t' \in [t - w/2, t + w/2]} \text{signal}(t', x, y)}{\text{max}_{t' \in [t - w/2, t + w/2]} \text{signal}(t', x, y) - \text{min}_{t' \in [t - w/2, t + w/2]} \text{signal}(t', x, y)}
\end{align}
$$

where $w$ is the window size.

Let's use a window size $w$ of 60 frames (120 ms) for this example and mask out the background.

In [ ]:
norm_warped = om.video.normalize_pixelwise_slidingwindow(warped, window_size=60)
norm_warped[:, background_mask] = np.nan

om.video.play(norm_warped, title="sliding window normalization", interval=20);

In [ ]:
norm_warped = om.video.normalize_pixelwise_slidingwindow(warped, window_size=60)
norm_warped[:, background_mask] = np.nan

render_ani_func(lambda: om.video.play(norm_warped, title="sliding window normalization", interval=20))

In [ ]:
alpha = om.video.normalize(norm_warped, vmin=0.5, vmax=0)
om.video.play_with_overlay(warped, 1-norm_warped, alpha=alpha, vmin_overlay=0, vmax_overlay=1, interval=30);

In [ ]:
alpha = om.video.normalize(norm_warped, vmin=0.5, vmax=0)
render_ani_func(lambda: om.video.play_with_overlay(warped, (1-norm_warped), alpha=alpha, interval=20))

### Temporal difference

The temporal difference approach computes the difference between frames at time $t$ and $t - n$:

$$
\begin{align}
    \text{signal}_{\text{diff}}(t, x, y) = \text{signal}(t, x, y) - \text{signal}(t - n, x, y)
\end{align}
$$

where $\Delta t$ is the time difference between frames. This method is more robust, however it does generally not conserve the shape of the action potential. Rather, it isolates the front of the fluorescent waves.

Let's use $n = 10$ for this example.

In [ ]:
n = 5
diff = om.video.temporal_difference(warped, n)
diff[:, background_mask] = np.nan
abs_max = 0.33*np.nanmax(np.abs(diff))
om.video.play(diff, title="temporal difference", cmap="PiYG", vmin=-abs_max, vmax=abs_max, interval=20);

In [ ]:
diff[diff > 0] = 0
om.video.play(diff, title="temporal difference", cmap="PiYG", vmin=-abs_max, vmax=abs_max, interval=20);

In [ ]:
om.video.play2(norm_warped, diff, title1="Sliding Window", title2="Frame Difference", vmin1=0, vmax1=1, cmap2='gray', vmin2=0.2*np.nanmin(diff), vmax2=0, interval=20);

In [ ]:
diff_norm = om.video.normalize_pixelwise_slidingwindow(diff, window_size=60)
om.video.play2(norm_warped, diff_norm, vmin1=0, vmax1=1, cmap2='gray', vmin2=0, vmax2=1, interval=20);

In [ ]:
diff = om.video.temporal_difference(warped, 10)
diff[diff > 0] = 0
diff[:, background_mask] = np.nan
diff_norm = om.video.normalize_pixelwise_slidingwindow(-diff, window_size=60)
om.video.play_with_overlay(warped, diff_norm, vmin_overlay=-1, vmax_overlay=1)

In [ ]:
diff = om.video.temporal_difference(warped, 10)
diff[diff > 0] = 0
diff[:, background_mask] = np.nan
diff_norm = om.video.normalize_pixelwise_slidingwindow(-diff, window_size=60)
render_ani_func(lambda: om.video.play_with_overlay(warped, diff_norm, vmin_overlay=-1, vmax_overlay=1, interval=20))